In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np 
import argparse
import pickle 
import os
from torch.autograd import Variable 
from torchvision import transforms 
from data_loader import build_vocab 
from model import EncoderCNN, DecoderRNN
from model import ResNet, ResidualBlock
from PIL import Image


embed_size = 256
hidden_size = 512 
num_layers = 1 
decoder_path = './models/decoder-5-150.pkl'
encoder_path = './models/encoder-5-150.pkl'


def to_var(x, volatile=False):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x, volatile=volatile)

def load_image(image_path, transform=None):
    image = Image.open(image_path).convert('RGB')
    image = image.resize([64, 64], Image.LANCZOS)
    
    if transform is not None:
        image = transform(image).unsqueeze(0)
    
    return image

transform = transforms.Compose([ 
    transforms.ToTensor(), 
    transforms.Normalize((0.033, 0.032, 0.033), 
                         (0.027, 0.027, 0.027))])



vocab_path = './data/vocab.pkl'
with open(vocab_path, 'rb') as f:
    vocab = pickle.load(f)
len_vocab = vocab.idx

# Build Models
encoder = ResNet(ResidualBlock, [3, 3, 3], len_vocab)
encoder.eval()  # evaluation mode (BN uses moving mean/variance)
decoder = DecoderRNN(len_vocab, hidden_size, 
                     len(vocab), num_layers)


# Load the trained model parameters
encoder.load_state_dict(torch.load(encoder_path))
decoder.load_state_dict(torch.load(decoder_path))

# If use gpu
if torch.cuda.is_available():
    encoder.cuda()
    decoder.cuda()

    

In [ ]:
test_dir = 'data/2object_test/bitmap/'
cap_dir = 'data/2object_test/caption/'
test_list = os.listdir(test_dir)
cnt = 0 

for fname in test_list: 
    test_path = test_dir + fname
    test_image = load_image(test_path, transform)
    image_tensor = to_var(test_image)
    
    # Generate caption from image
    feature = encoder(image_tensor)
    sampled_ids = decoder.sample(feature)
    ids_arr = []
    for element in sampled_ids: 
        temp = element.cpu().data.numpy()
        ids_arr.append(int(temp))


    # Decode word_ids to words
    sampled_caption = []
    for word_id in ids_arr:
        word = vocab.idx2word[word_id]
        sampled_caption.append(word)
        if word == '<end>':
            break
    sentence = ' '.join(sampled_caption)
    print(sentence)
    predict = sentence.split()[1]
    
    cap_target = cap_dir+fname
    cap_path = cap_target.split('.')[0]+'.svg'
    with open(cap_path, 'r') as f:
        target = f.readline()
        print('target: '+ target)
    if predict == target:
        print('corret')
    #    cnt+=1 
        
print(cnt)
    
    
    

In [ ]:
feature.unsqueeze(1)

In [ ]:
cap_path = img_list[0].split('.')[0]+'.svg'

In [ ]:
cap_path